In [2]:
!pip install -q qiskit
!pip install -q qiskit_aer
!pip install pylatexenc

In [11]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from math import gcd, log2, pi
from fractions import Fraction
from random import randint

In [12]:
# Define a function to perform Quantum Fourier Transform
def qft(circuit, qubits):
    n= len(qubits)

    # Perform Hadamard and controlled phase rotations
    for i in range(n):
        circuit.h(qubits[i])
        for k in range(i+1, n):
            circuit.cp(pi/(2**(k - i)), qubits[k], qubits[i])

    # Reverse the order of the qubits
    for i in range(n//2):
        circuit.swap(qubits[i], qubits[n - i - 1])

In [13]:
# Define a function to perform Inverse Quantum Fourier Transform
def iqft(circuit, qubits):
    n= len(qubits)

    # Perform the inverse of the operations applied during the Quantum Fourier Transform (in reverse order)
    for i in range(n//2):
        circuit.swap(qubits[i], qubits[n - i - 1])

    for i in reversed(range(n)):
        for k in reversed(range(i)):
            circuit.cp(-pi/(2**(i - k)), qubits[k], qubits[i])
        circuit.h(qubits[i])

In [14]:
# Define a function to perform phase estimation for finding the order of a mod N in Shor's Algorithm
def phase_estimation(circuit, control, target, a, N):
    n = len(control)

    # Apply Hadamard to control
    circuit.h(control)

    # Initialize target in |1⟩
    circuit.x(target[0])

    # Fake modular exponentiation loop (for visual/simulated behavior only)
    for j in range(n):
        mod = pow(a, 2**j, N)
        if mod == 0:
            continue  # Skip log2(0)
        try:
            i = int(log2(mod))
            if i < len(target):
                circuit.cx(control[j], target[i])
        except:
            continue  # Safe fallback

    iqft(circuit, control)

In [24]:
# Factor 1591 = 37 x 43 using Shor's Algorithm
flag= False
N = 1591
while flag is False: # repeat until non- trivial factors are found
    a= randint(2, N-1)
    if gcd(a, N)!= 1:
        continue

    control= QuantumRegister(12) # 12 qubits for a high precision
    target= QuantumRegister(11) #11 qubits to represent all numbers from 0 to N-1
    c= ClassicalRegister(12)
    circuit= QuantumCircuit(control, target, c)

    phase_estimation(circuit, control, target, a, N)
    circuit.measure(control, c)

    counts= AerSimulator().run(circuit, shots=1000).result().get_counts()
    measured= max(counts, key=counts.get)
    phase= int(measured, 2)/(2**12) # convert the binary string to a phase value
    r= Fraction(phase).limit_denominator(N).denominator # use continued fractions to estimate r from the measured phase

    if r%2== 1: # retry if r is odd
        continue

    p= gcd(pow(a, r//2)- 1, N)
    q= gcd(pow(a, r//2)+ 1, N)

    if p== 1 or q== 1 or p== N or q== N: # retry if the factors are non- trivial
        continue

    print(f"The Prime Factors of {N} are {p} and {q}")
    flag= True


The Prime Factors of 1591 are 43 and 37
